In [2]:
from PIL import Image
import PySimpleGUI as sg
import io


image = Image.open('bankas.png')
with io.BytesIO() as bio:
    image.save('bankas.png', format="PNG")
    image_data = bio.getvalue()


def calculate_net_worth(assets):
    return sum(asset['price'] for asset in assets)

def calculate_passive_income(assets):
    monthly_income = 0
    for asset in assets:
        monthly_return = asset['price'] * (asset['yearly_return'] / 100) / 12
        monthly_income += monthly_return
        if asset['reinvest_profit']:
            asset['price'] += monthly_return  # Reinvest the monthly return back into the asset
    return monthly_income



sg.theme('DarkBlack') 
# Define the layout for the GUI
layout = [ 
    [sg.Image(size=(640, 240), filename= 'bankas.png', key= 'background')],  
    [sg.Text("Asset Manager", size=(30, 1), font=("Helvetica", 25), text_color='blue')],
    [sg.Text("Add your assets and details below")],
    [sg.Text("Asset Name"), sg.InputText(key='asset_name', justification="center", size=(15, 10))],
    [sg.Text("Price"), sg.InputText(key='price', justification="right", size=(21, 10))],
    [sg.Text("Yearly Return (%) "), sg.InputText(key='yearly_return', justification="right", size=(10, 10))],
    [sg.Checkbox("Reinvest Profit", key='reinvest_profit')],
    [sg.Button("Add Asset"), sg.Button("Remove Asset"), sg.Button("Clear List")],
    [sg.Listbox(values=[], size=(33, 6), key='asset_list')],
    [sg.Text("Total Net Worth:"), sg.Text('0', size=(20, 1), key='total_net_worth')],
    [sg.Text("Net Worth Target:"), sg.InputText('10000', key='net_worth_target', justification="center", size=(17, 10))],
    [sg.ProgressBar(max_value=100, orientation='h', size=(22, 20), key='progress_bar')],
    [sg.Text("Estimated Passive Income Next Month:"), sg.Text('$0', size=(13, 1), key='passive_income')],
    [sg.Exit()]
]

window = sg.Window("Raudona skaiciuokle", layout, resizable=True)


assets = []

while True:
    event, values = window.read()

    # If user closes window or clicks 'Exit', end the program
    if event in (sg.WIN_CLOSED, 'Exit'):
        break

    if event in ["Add Asset", "Remove Asset", "Clear List"]:
        # Asset operations
        if event == "Add Asset":
            try:
                asset_details = {
                    'name': values['asset_name'],
                    'price': float(values['price']),
                    'yearly_return': float(values['yearly_return']),
                    'reinvest_profit': values['reinvest_profit']
                }
                assets.append(asset_details)
            except ValueError:
                sg.popup_error("Please enter valid numbers for price and yearly return.")
        if event == "Remove Asset":
            if values['asset_list']:  # Check if an item is selected
                selected_asset = values['asset_list'][0]
                # Match the 'name' of the asset to remove with the selected list item
                # assets = [asset for asset in assets if asset['name'] != selected_asset[0]]
                selected_name, selected_price = selected_asset[0], float(selected_asset[1])
                assets = [asset for asset in assets if not (asset['name'] == selected_name and asset['price'] == selected_price)]



        if event == "Clear List":
            assets.clear()

        # Update the Listbox with the new asset list
        window['asset_list'].update(values=[(a['name'], a['price']) for a in assets])
        
        # Calculate and display total net worth
        total_net_worth = calculate_net_worth(assets)
        window['total_net_worth'].update(f"${total_net_worth:.2f}")
        
        # Calculate and display the estimated passive income for the next month
        passive_income = calculate_passive_income(assets)
        window['passive_income'].update(f"${passive_income:.2f}")

window.close()

